Licensed under the MIT License.

Copyright (c) 2021-2025. All rights reserved.

# Keras Tuners Comparison

* Using CNN
* Flowers102 Data
* Tuners built in Keras Tuner: https://keras.io/api/keras_tuner/tuners/
* Keras hyperparameters (hp) methods: https://keras.io/api/keras_tuner/hyperparameters/
* Tuner class: https://keras.io/api/keras_tuner/tuners/base_tuner/#tuner-class

In [1]:
import numpy as np

import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping

tf.keras.backend.clear_session()
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
print("Num GPUs:", len(physical_devices))

2.6.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs: 1


In [2]:
X_train = np.load('../../crystal_ball/data_collector/structured_data/flowers_X_train_norm.npy')
X_test = np.load('../../crystal_ball/data_collector/structured_data/flowers_X_test_norm.npy')
y_train = np.load('../../crystal_ball/data_collector/structured_data/flowers_y_train.npy')
y_test = np.load('../../crystal_ball/data_collector/structured_data/flowers_y_test.npy')

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(7169, 300, 300, 3) (7169, 1)
(1020, 300, 300, 3) (1020, 1)


In [3]:
N_TRAIN_EXAMPLES = X_train.shape[0]
N_VALID_EXAMPLES = X_test.shape[0]
BATCHSIZE = 32
CLASSES = 102
VAL_SPLIT = 0.2
EPOCHS = 3

img_x = X_train.shape[1]
img_y = X_train.shape[2]
channel = X_train.shape[3]
input_shape = (img_x, img_y, channel)

In [6]:
def build_model(hp):
    clear_session()  # clear clutter from previous tf.keras session graphs
    
    model = Sequential()
    model.add(
        Conv2D(
            filters=hp.Choice("filters", [64, 128]),
            kernel_size=hp.Choice("kernel_size", [3, 5]),
            strides=hp.Choice("strides", [1, 2, 3]),
            activation=hp.Choice("activation", ["relu", "sigmoid", "tanh"]),
            input_shape=input_shape,
        )
    )
    model.add(Flatten())
    model.add(Dense(CLASSES, activation="softmax"))

    learning_rate = hp.Float("learning_rate", min_value=1e-5, max_value=1e-3, sampling='log')
    
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=RMSprop(learning_rate=learning_rate), metrics=["accuracy"]
    )
    
    return model


def get_optimized_model(my_tuner, callbacks=None):
    """
    1. Get best params through the search
    2. Use the best params to retrain the whole training data to get the best epoch
    3. Ues the best epoch & best params to retrain the whole training data to get the final optimized model, which cna be used to evaluate the test data
    """
    print('Searching for best params...')
    if callbacks != None:
        my_tuner.search(X_train, y_train, epochs=EPOCHS, validation_split=VAL_SPLIT, callbacks=callbacks)
    else:
        my_tuner.search(X_train, y_train, epochs=EPOCHS, validation_split=VAL_SPLIT)
    print()
    
    
    print('Searching for best epoch...')
    best_hps = my_tuner.get_best_hyperparameters(num_trials=1)[0]
    model = my_tuner.hypermodel.build(best_hps)
    history = model.fit(X_train, y_train, epochs=EPOCHS, validation_split=VAL_SPLIT, batch_size=BATCHSIZE)

    val_acc_per_epoch = history.history['val_accuracy']
    best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
    print(f'Best epoch: {best_epoch}')
    print()
        
        
    print('Retraining model with best params & best epoch...')
    opt_model = my_tuner.hypermodel.build(best_hps)
    opt_model.fit(X_train,y_train, epochs=best_epoch, validation_split=VAL_SPLIT)
    print()
        
    return opt_model, my_tuner

## Random Search Tuner

In [5]:
with tf.device("/cpu:0"):
    rs_tuner = kt.RandomSearch(build_model,
                               objective='val_accuracy',
                               max_trials=2,
                               seed=10,
                               project_name='kt_rs',
                               overwrite=True)

    rs_model, rs_tuner = get_optimized_model(rs_tuner)
    print('Tuner Summary: ')
    print(rs_tuner.results_summary())
    print()
    
    print('Model Summary:')
    rs_model.summary()
    print()
    
    eval_result = rs_model.evaluate(X_test, y_test)
    print(f'Test Loss: {eval_result[0]}, Test Accuracy: {eval_result[1]}')
    print()

Trial 2 Complete [00h 10m 07s]
val_accuracy: 0.38145047426223755

Best val_accuracy So Far: 0.38145047426223755
Total elapsed time: 00h 29m 16s
INFO:tensorflow:Oracle triggered exit

Searching for best epoch...
Epoch 1/3
180/180 [==============================] - 190s 1s/step - loss: 4.2300 - accuracy: 0.2014 - val_loss: 2.8269 - val_accuracy: 0.3264
Epoch 2/3
180/180 [==============================] - 194s 1s/step - loss: 1.2263 - accuracy: 0.7050 - val_loss: 3.2523 - val_accuracy: 0.2538
Epoch 3/3
180/180 [==============================] - 190s 1s/step - loss: 0.3392 - accuracy: 0.9322 - val_loss: 2.9539 - val_accuracy: 0.3508
Best epoch: 3

Retraining model with best params & best epoch...
Epoch 1/3
180/180 [==============================] - 178s 984ms/step - loss: 4.1649 - accuracy: 0.2065 - val_loss: 3.1128 - val_accuracy: 0.2120
Epoch 2/3
180/180 [==============================] - 176s 979ms/step - loss: 1.2613 - accuracy: 0.6987 - val_loss: 3.2005 - val_accuracy: 0.2922
Epoch 3/

## Bayesian Optimization

In [7]:
with tf.device("/cpu:0"):
    by_tuner = kt.BayesianOptimization(build_model,
                                       objective='val_accuracy',
                                       max_trials=2,
                                       num_initial_points=2,
                                       alpha=0.0001,
                                       beta=2.6,
                                       seed=10,
                                       project_name='kt_bayes',
                                       overwrite=True)

    by_model, by_tuner = get_optimized_model(by_tuner)
    print('Tuner Summary: ')
    print(by_tuner.results_summary())
    print()
    
    print('Model Summary:')
    by_model.summary()
    print()
    
    eval_result = by_model.evaluate(X_test, y_test)
    print(f'Test Loss: {eval_result[0]}, Test Accuracy: {eval_result[1]}')
    print()

Trial 2 Complete [00h 07m 21s]
val_accuracy: 0.35564854741096497

Best val_accuracy So Far: 0.35564854741096497
Total elapsed time: 00h 24m 24s
INFO:tensorflow:Oracle triggered exit

Searching for best epoch...
Epoch 1/3
180/180 [==============================] - 152s 844ms/step - loss: 3.8250 - accuracy: 0.2220 - val_loss: 3.0376 - val_accuracy: 0.2594
Epoch 2/3
180/180 [==============================] - 151s 840ms/step - loss: 1.2047 - accuracy: 0.7158 - val_loss: 2.6605 - val_accuracy: 0.3654
Epoch 3/3
180/180 [==============================] - 149s 830ms/step - loss: 0.3566 - accuracy: 0.9278 - val_loss: 2.6191 - val_accuracy: 0.3591
Best epoch: 2

Retraining model with best params & best epoch...
Epoch 1/2
180/180 [==============================] - 170s 939ms/step - loss: 4.3035 - accuracy: 0.2099 - val_loss: 2.8541 - val_accuracy: 0.3264
Epoch 2/2
180/180 [==============================] - 202s 1s/step - loss: 1.1636 - accuracy: 0.7320 - val_loss: 2.7724 - val_accuracy: 0.3522

T

## Hyperband Tuner

In [8]:
with tf.device("/cpu:0"):
    hb_tuner = kt.Hyperband(build_model,
                            objective='val_accuracy',
                            max_epochs=10,  # better to be larger than expected EPOCHS, and apply EarlyStopping together
                            factor=12,  # used to reduce the number of epoches and the number of models for each bracket
                            hyperband_iterations=1,  # one iteration will run approximately max_epochs * (math.log(max_epochs, factor) ** 2) cumulative epochs across all trials.
                            seed=10,
                            project_name='kt_hb',
                            overwrite=True)
    
    stop_early = EarlyStopping(monitor='val_loss', patience=5)

    hb_model, hb_tuner = get_optimized_model(hb_tuner, callbacks=[stop_early])
    print('Tuner Summary: ')
    print(hb_tuner.results_summary())
    print()
    
    print('Model Summary:')
    hb_model.summary()
    print()
    
    eval_result = hb_model.evaluate(X_test, y_test)
    print(f'Test Loss: {eval_result[0]}, Test Accuracy: {eval_result[1]}')
    print()

Trial 2 Complete [00h 22m 31s]
val_accuracy: 0.4044630527496338

Best val_accuracy So Far: 0.4044630527496338
Total elapsed time: 01h 30m 48s
INFO:tensorflow:Oracle triggered exit

Searching for best epoch...
Epoch 1/3
180/180 [==============================] - 192s 1s/step - loss: 4.2686 - accuracy: 0.2030 - val_loss: 2.9427 - val_accuracy: 0.2789
Epoch 2/3
180/180 [==============================] - 189s 1s/step - loss: 1.2350 - accuracy: 0.7050 - val_loss: 2.7053 - val_accuracy: 0.3501
Epoch 3/3
180/180 [==============================] - 191s 1s/step - loss: 0.3564 - accuracy: 0.9259 - val_loss: 2.7371 - val_accuracy: 0.3480
Best epoch: 2

Retraining model with best params & best epoch...
Epoch 1/2
180/180 [==============================] - 188s 1s/step - loss: 4.2317 - accuracy: 0.1948 - val_loss: 3.2663 - val_accuracy: 0.2434
Epoch 2/2
180/180 [==============================] - 188s 1s/step - loss: 1.3468 - accuracy: 0.6792 - val_loss: 3.1402 - val_accuracy: 0.2734

Tuner Summary: 